# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score

data = pd.read_csv('/datasets/toxic_comments.csv')
data.info()
data.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
110992,If Music Be the Food of Love... Then Prepare f...,0
76494,Gil Elvgren Tribute \n\nI was just watching th...,0
47429,Thank you very much! ),0
49486,Your Advisor\nA little bit more context for th...,0
98028,Notability \n\nHi there; you have unerringly s...,0


In [2]:
data['text'] = data['text'].str.lower()
data['text'][0]

"explanation\nwhy the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.89.205.38.27"

In [3]:
if data.duplicated().sum() != 0:
    data = data.drop_duplicates()

print('Количество строк на класс: ', np.bincount(data['toxic']))
data['toxic'].value_counts(normalize=True)

Количество строк на класс:  [143316  16210]


0    0.898386
1    0.101614
Name: toxic, dtype: float64

Выборка несбалансирована. Возможно стоит применить downsampling на такой большой выборке?

In [4]:
import re
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import EnglishStemmer

nltk.download('averaged_perceptron_tagger')


def clean_token(text):
    
    text = re.sub(r'[^a-z\' ]', ' ', text)
    tokens = nltk.word_tokenize(text)
    
    return tokens


def make_lemm(text):
    
    lemmer = WordNetLemmatizer()
    tag_tokens = nltk.pos_tag(clean_token(text))
    
    sentence = [lemmer.lemmatize(token, 
                                 pos=((tag[0].lower() if tag[0].lower() in 'nvr' else 'n') if tag[0].lower()!='j' else 's')) 
                for token, tag in tag_tokens]
    
    return ' '.join(sentence)


def make_stem(text):
    
    stemmer = EnglishStemmer()
    sentence = [stemmer.stem(w) for w in clean_token(text)]
    
    return ' '.join(sentence)


print(make_stem(data['text'][0]))
make_lemm(data['text'][0])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


explan whi the edit made under my usernam hardcor metallica fan were revert they were n't vandal just closur on some gas after i vote at new york doll fac and pleas do n't remov the templat from the talk page sinc i 'm retir now


"explanation why the edits make under my username hardcore metallica fan be revert they be n't vandalisms just closure on some gas after i vote at new york doll fac and please do n't remove the template from the talk page since i 'm retire now"

In [5]:
%%time
target = data['toxic']
feat = data['text'].apply(make_stem)
len(feat)

CPU times: user 3min 35s, sys: 792 ms, total: 3min 36s
Wall time: 3min 37s


159526

In [6]:
%%time
feat_lemm = data['text'].apply(make_lemm)
len(feat_lemm)

CPU times: user 12min 6s, sys: 6.73 s, total: 12min 13s
Wall time: 12min 14s


159526

Лемматизация долгая из-за кучи условий, а так она даже быстрее чем стемминг в 2 раза была.

---
В tf-idf можно же отбросить редко встречающиеся слова? Хотя, судя по несбалансированности, это как раз могут оказаться слова-признаки указывающие на токсичность. 

Стоит посмотреть на сами токсичные комментарии:

In [7]:
data[data['toxic'] == 1]['text'].sample(5).values.astype('U')

array(['"\nlook, the anglophobic vigilante has come bearing ill news and ""ill news is an ill guest"".  watch out, tyke, he considers himself to have inviolate powers of edit warring upon your contributions, now that he sees some other people have an unrelated dispute with you.  all the more ""justification"" for him to run roughshod over you to get what he wants.  it\'s been done before.  just check at my own edits for proof.    "',
       "dickhead\nyou know it was stupid, you know it wasn't clever. so why do it? 94.195.251.61",
       "u suck!!!!\nu suck!!!!'u suck!!!!u suck!!!!'u suck!!!!\nu suck!!!!'u suck!!!!u suck!!!!'u suck!!!!\nu suck!!!!'u suck!!!!u suck!!!!'u suck!!!!\nu suck!!!!'u suck!!!!u suck!!!!'u suck!!!!\nu suck!!!!'u suck!!!!u suck!!!!'u suck!!!!\nu suck!!!!'u suck!!!!u suck!!!!'u suck!!!!\nu suck!!!!'u suck!!!!u suck!!!!'u suck!!!!\nu suck!!!!'u suck!!!!u suck!!!!'u suck!!!!\nu suck!!!!'u suck!!!!u suck!!!!'u suck!!!!\nu suck!!!!'u suck!!!!u suck!!!!'u suck!!!!\nu s

Tf-idf вместо 5000 слов! Tf для слова fucksex будет 0.99, а idf - примерно 4.2 (скорее всего это редкое слово и более чем в 10 документах его не будет). Tf-idf ~ 4.16, вот такие слова, с высоким значением tf-idf тут и нужны (это еще раз к тому что downsampling плохая идея). Несбалансированность тут как раз и создает признак.

Исходя из текстов могут быть полезны N-граммы от 1 до 4 
- Что если просто ограничить число фичей?
- А если попробовать отобрать признаки из этой кучи по какоиму нибудь правилу? 

# 2. Обучение

In [8]:
%%time

from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.feature_selection import chi2, SelectKBest, f_classif



def tf_idf_vect(feat, target):
    
    stp_wrds = set(stopwords.words('english'))
    
    tf_idf = TfidfVectorizer(min_df=2, max_df=0.9, max_features=500000, 
                             ngram_range=(1, 4), sublinear_tf=True,
                             stop_words=stp_wrds)
    select = SelectKBest(chi2, k=100000)
    
    feat_train, feat_test, y_train, y_test = train_test_split(feat, target, test_size=0.33, random_state=42)
    
    tf_train = tf_idf.fit_transform(feat_train)
    tf_test = tf_idf.transform(feat_test)
    
    tf_train_new = select.fit_transform(tf_train, y_train)
    tf_test_new = select.transform(tf_test)
    
    return tf_train_new, tf_test_new, y_train, y_test



tf_train, tf_test, y_train, y_test = tf_idf_vect(feat, target)
train_lemm, test_lemm, y_train, y_test = tf_idf_vect(feat_lemm, target)

print('Стеммы:', tf_train.shape, y_train.shape, tf_test.shape, y_test.shape)
print('Леммы:', train_lemm.shape, y_train.shape, test_lemm.shape, y_test.shape)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Стеммы: (106882, 100000) (106882,) (52644, 100000) (52644,)
Леммы: (106882, 100000) (106882,) (52644, 100000) (52644,)
CPU times: user 2min 48s, sys: 6.67 s, total: 2min 55s
Wall time: 2min 56s


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB

pipe = Pipeline([('clf', LogisticRegression())])

pipe_grid = [
    {'clf': [LogisticRegression(solver='sag', max_iter=5000, 
                                class_weight='balanced', random_state=42)],
     'clf__C':             [1, 5, 10, 12]
    }, 
    {'clf': [SGDClassifier(class_weight='balanced', max_iter=5000, learning_rate='adaptive', 
                           early_stopping=True, eta0=0.5, random_state=42)],
     'clf__loss':          ['modified_huber', 'squared_hinge'],
     'clf__eta0':          [0.5, 0.1, 1, 2, 3, 4]
    },
    {'clf': [MultinomialNB()],
     'clf__alpha':         [0.6, 0.3, 0.1, 1],
     'clf__class_prior':   [[1, 2], None]
    }
         ]

def search_model(pipe, pipe_grid, X, y, test, y_test):
    
    grid_search = GridSearchCV(pipe, pipe_grid, cv=5, scoring='f1')
    grid_search.fit(X, y)
    
    best_estim = grid_search.best_estimator_.named_steps['clf']
    
    print('Лучшая модель:\n', best_estim)
    print('F1 Score: ', grid_search.best_score_)
    print('Test F1 Score: ', grid_search.score(test, y_test))
    
    cv_reslt = pd.DataFrame(grid_search.cv_results_
                           ).sort_values(by='rank_test_score')
    
    return cv_reslt[
        [col for col in cv_reslt.columns 
         if 'param_' in col] + 
        ['mean_fit_time', 'mean_test_score']
                   ], best_estim

In [10]:
%%time
print('На леммах:')
cv_result, best_estim = search_model(pipe, pipe_grid, train_lemm, y_train, test_lemm, y_test)
cv_result

На леммах:
Лучшая модель:
 LogisticRegression(C=12, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=5000, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)
F1 Score:  0.7891661234045364
Test F1 Score:  0.7747631748367516
CPU times: user 10min 34s, sys: 6.62 s, total: 10min 40s
Wall time: 10min 43s


,param_clf,param_clf__C,param_clf__eta0,param_clf__loss,param_clf__alpha,param_clf__class_prior,mean_fit_time,mean_test_score
3,"LogisticRegression(C=12, class_weight='balance...",12,NaN,NaN,NaN,NaN,29.261751,0.789166
2,"LogisticRegression(C=12, class_weight='balance...",10,NaN,NaN,NaN,NaN,18.155937,0.789107
1,"LogisticRegression(C=12, class_weight='balance...",5,NaN,NaN,NaN,NaN,20.268415,0.787378
6,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,0.1,modified_huber,NaN,NaN,2.325961,0.770786
7,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,0.1,squared_hinge,NaN,NaN,2.361076,0.770726
11,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,2,squared_hinge,NaN,NaN,2.841924,0.770713
10,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,2,modified_huber,NaN,NaN,2.856032,0.770651
5,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,0.5,squared_hinge,NaN,NaN,2.556110,0.770649
13,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,3,squared_hinge,NaN,NaN,2.857775,0.770606
12,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,3,modified_huber,NaN,NaN,2.864246,0.770558


In [11]:
%%time
print('На стеммах:')
cv_sresult, sbest_estim = search_model(pipe, pipe_grid, tf_train, y_train, tf_test, y_test)
cv_sresult

На стеммах:
Лучшая модель:
 LogisticRegression(C=12, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=5000, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)
F1 Score:  0.79695414199236
Test F1 Score:  0.7869817045141123
CPU times: user 10min 2s, sys: 6.3 s, total: 10min 8s
Wall time: 10min 9s


,param_clf,param_clf__C,param_clf__eta0,param_clf__loss,param_clf__alpha,param_clf__class_prior,mean_fit_time,mean_test_score
3,"LogisticRegression(C=12, class_weight='balance...",12,NaN,NaN,NaN,NaN,17.556702,0.796954
2,"LogisticRegression(C=12, class_weight='balance...",10,NaN,NaN,NaN,NaN,27.541810,0.795836
1,"LogisticRegression(C=12, class_weight='balance...",5,NaN,NaN,NaN,NaN,21.706217,0.794496
11,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,2,squared_hinge,NaN,NaN,2.849763,0.780905
9,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,1,squared_hinge,NaN,NaN,2.631774,0.780720
10,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,2,modified_huber,NaN,NaN,2.875435,0.780680
12,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,3,modified_huber,NaN,NaN,2.822159,0.780575
8,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,1,modified_huber,NaN,NaN,2.631151,0.780528
14,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,4,modified_huber,NaN,NaN,2.820902,0.780434
15,"SGDClassifier(alpha=0.0001, average=False, cla...",NaN,4,squared_hinge,NaN,NaN,2.766557,0.780430


In [12]:
#Блендинг?

model_2 = SGDClassifier(class_weight='balanced', max_iter=5000, 
                        learning_rate='adaptive', early_stopping=True, 
                        eta0=2, random_state=42, loss='squared_hinge')
model_2.fit(tf_train, y_train)

f1_score(y_test, 
        ((sbest_estim.predict(tf_test) 
          + 
          model_2.predict(tf_test)
         )*.5).round())

0.7872478854912166

----
### Теперь можно попробовать BERT

In [24]:
import torch
import transformers as ppb
from tqdm import notebook

#готовенький берт из коробочки
tokenizer = ppb.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model_bert = ppb.BertModel.from_pretrained('bert-base-uncased')

#Попробую хотя бы на половине данных.
df_bert1, df_bert2 = train_test_split(data, test_size=0.5, random_state=42)
df_bert1.shape

(79785, 2)

In [27]:
%%time
# токенизируем текст, максимальную длинну вектора фиксируем на 300, т.к. эта модель не вынесет больше 512.
max_len = 300       
tokenized = df_bert1['text'].apply(lambda x: tokenizer.encode(x, max_length=max_len, add_special_tokens=True))
# max_len = max([len(i) for i in tokenized])

# применим padding к векторам
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

CPU times: user 3min 8s, sys: 1.62 s, total: 3min 10s
Wall time: 3min 12s


In [ ]:
batch_size = 100                                                            # размер батча.
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):               # Это количество эпох
    
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])     # Батч, буквально срез вектора.
        attention_mask_batch = torch.LongTensor(                            # LongTensor это тип данных, а можно не лонг?
            attention_mask[batch_size*i:batch_size*(i+1)])                  # маска вектора, теперь маска батча
        
        with torch.no_grad():                                               # ??? типа отрубили автоградиент?
            batch_embeddings = model_bert(                                  # смотрим слова эмбединги
                batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())               # что в остальных?

feat_bert = np.concatenate(embeddings)
feat_bert.shape

ориентировочное время завершения 50 часов и ядро постоянно падает... терпиливым быть конечно хорошо, но всему есть предел)

И у меня совершенно нет идей как ускорить процесс эмбеддинга...

In [ ]:
from sklearn.model_selection import cross_val_score

X_train, X_test, y_train, y_test = train_test_split(feat_bert, df_bert1['toxic'], test_size=0.33, random_state=42)

model_clf = LogisticRegression(random_state=42, solver='sag')
print('F1 Cross_val_score: ', cross_val_score(model_clf, X_train, y_train, cv=5, scoring='f1').mean())
print('F1 Test score: ', f1_score(model_clf.predict(X_test), y_test))

# 3. Выводы

- Все очень сильно зависит от предобработки и полученния признаков из tf-idf

Хотя разница в точности моделей на леммах и стеммах несущественная, зато лучшее время показывает лемматизация без тэгов и фиксированным парт'оф'спич, почти в 2 раза быстрее стемминга.

При увеличении кол-ва признаков некоторые модели теряли в качестве почти в 2 раза. Заоблачные количества признаков на N граммах тоже удивили. Биграмм под 2кк, а триграмм аж 3кк... Мне казалось их должно становиться меньше, или он все со всеми склеивает? Может это потому что он стоп-слова не выкидывает?
- Пробовал downsampling. 

Использовать с tf-idf нет смысла, upsempling тем более. Хотя, если применять его ко всему корпусу, то результат положительный, но это неправильно, просто выкинуть 30% данных с 0 классом... 

Без downsampling'а самая лучшая модель с балансировкой классов давала на трейне ~0.75, на тесте 0.7469. С ним, примененным только к трейну 0.792, а на тесте 0.739, уже похоже на переобучение. С ним, примененным ко всему датасету, на трейне примерно также 0.79, зато на тесте уже около 0.77.

- Наивныей Байес Бернулли замечательно обучается, но на тесте у него ничего не выходит. Похоже на переобучение. Поэтому взял MultinomialNB, чтобы в топ не выскакивал.
- Дерево и Лес не стал использовать т.к. это занимает очень много времени, результат скорее всего будет немногим лучше, а в случае с деревом возможно и хуже.
- Лучшего токенайзера для английского чем split() я не нашел. Не мог решить проблему "don't" -> 'do', "n't". На самом деле без разницы, они все равно должны были попасть в стоп-слова, хотя на N-граммах это могло отразиться 2 лишними итерациями.
- Bert. Bert штука мощная. С такой в тренажере особо не развернешься.